In [13]:
from openai import OpenAI, RateLimitError, APIError, APITimeoutError
import pandas as pd 
from tqdm.notebook import tqdm
from dotenv import load_dotenv
import re
import numpy as np
import json
import argparse
import random
import time
import os
import ast

In [14]:
%cd "/Users/sm9518/Library/CloudStorage/GoogleDrive-sm9518@princeton.edu/My Drive/Classes/Stats-blog/posts/final-project"

/Users/sm9518/Library/CloudStorage/GoogleDrive-sm9518@princeton.edu/My Drive/Classes/Stats-blog/posts/final-project


/Users/sm9518/Library/Python/3.9/lib/python/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [15]:
load_dotenv("/Users/sm9518/Desktop/Article-Summarizer/.env") # where i keep my API key... 
api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print("API Key loaded successfully!\n:)")
else:
    raise ValueError("API Key not found.\nMake sure it is set in the .env file.")
model="gpt-3.5-turbo" # set model. we dont need anything fancy for this task.
temperature=0 # set temp to be rather determinisitic 
SEED = random.seed(42) # set seed for reproducibility

API Key loaded successfully!
:)


In [16]:
df = pd.read_csv('/Users/sm9518/Library/CloudStorage/GoogleDrive-sm9518@princeton.edu/My Drive/Classes/Stats-blog/posts/final-project/data/pokedex.csv', index_col=0)
df.head()

,name,height,weight,hp,attack,defense,s_attack,s_defense,speed,type,evo_set,info
id,,,,,,,,,,,,
1,bulbasaur,7,69,45,49,49,65,65,45,"{grass,poison}",1,A strange seed was planted on its back at birt...
2,ivysaur,10,130,60,62,63,80,80,60,"{grass,poison}",1,"When the bulb on its back grows large, it appe..."
3,venusaur,20,1000,80,82,83,100,100,80,"{grass,poison}",1,The plant blooms when it is absorbing solar en...
4,charmander,6,85,39,52,43,60,50,65,{fire},2,"Obviously prefers hot places. When it rains, s..."
5,charmeleon,11,190,58,64,58,80,65,80,{fire},2,"When it swings its burning tail, it elevates t..."


In [17]:
OG_pokedex = df.iloc[:151].copy() # take the OG 151 pokemon

# build the 20‐matchups per Pokémon
matchups = []
for challenger in OG_pokedex['name']:
    pool = [p for p in OG_pokedex['name'] if p != challenger]
    opponents = random.sample(pool, 20) # give them 20 challengers
    for opponent in opponents:
        matchups.append({'challenger': challenger, 'opponent': opponent})
matchups_df = pd.DataFrame(matchups)



# merge challenger metadata
matchups_with_meta = (
    matchups_df
    .merge(
        OG_pokedex.add_suffix('_challenger'),
        left_on='challenger',
        right_on='name_challenger',
        how='left'
    )
    # drop the redundant name_challenger column if you like
    .drop(columns=['name_challenger'])
    # merge opponent metadata
    .merge(
        OG_pokedex.add_suffix('_opponent'),
        left_on='opponent',
        right_on='name_opponent',
        how='left'
    )
    .drop(columns=['name_opponent'])
)

# now every row has both challenger_* and opponent_* columns
matchups_with_meta.head()

,challenger,opponent,height_challenger,weight_challenger,hp_challenger,attack_challenger,defense_challenger,s_attack_challenger,s_defense_challenger,speed_challenger,...,weight_opponent,hp_opponent,attack_opponent,defense_opponent,s_attack_opponent,s_defense_opponent,speed_opponent,type_opponent,evo_set_opponent,info_opponent
0,bulbasaur,nidorina,7,69,45,49,49,65,65,45,...,200,70,62,67,55,55,56,{poison},12,The female's horn develops slowly. Prefers phy...
1,bulbasaur,wartortle,7,69,45,49,49,65,65,45,...,225,59,63,80,65,80,58,{water},3,Often hides in water to stalk unwary prey. For...
2,bulbasaur,tentacool,7,69,45,49,49,65,65,45,...,455,40,40,35,50,100,70,"{water,poison}",30,Drifts in shallow seas. Anglers who hook them ...
3,bulbasaur,kadabra,7,69,45,49,49,65,65,45,...,565,40,35,30,120,70,105,{psychic},27,It emits special alpha waves from its body tha...
4,bulbasaur,arcanine,7,69,45,49,49,65,65,45,...,1550,90,110,80,100,80,95,{fire},25,A POKéMON that has been admired since the past...


In [47]:
# randomly sample 10% to make sure it works before deploying at scale
#matchups_with_meta = matchups_with_meta.sample(frac=0.1,random_state=SEED)
#matchups_with_meta.shape

(302, 24)

In [19]:
# Initialize OpenAI client
client = OpenAI()
# ---- Utility Functions ---- #

def safe_parse_types(val):
    if isinstance(val, list):
        return val
    try:
        return ast.literal_eval(val)
    except Exception:
        return [str(val)]

def format_pokemon_stats(name, row, suffix):
    types = safe_parse_types(row[f'type{suffix}'])
    return (
        f"{name.title()}:\n"
        f"- Type: {', '.join(types)}\n"
        f"- HP: {row[f'hp{suffix}']}\n"
        f"- Attack: {row[f'attack{suffix}']}\n"
        f"- Defense: {row[f'defense{suffix}']}\n"
        f"- Special Attack: {row[f's_attack{suffix}']}\n"
        f"- Special Defense: {row[f's_defense{suffix}']}\n"
        f"- Speed: {row[f'speed{suffix}']}\n"
    )

# ---- API Interaction ---- #

def get_completion(prompt):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    return response.choices[0].message.content.strip()

def get_response(prompt):
    try:
        return get_completion(prompt)
    except RateLimitError as e:
        retry_time = getattr(e, 'retry_after', 30)
        print(f"Rate limit exceeded. Retrying in {retry_time} seconds...")
        time.sleep(retry_time)
        return get_response(prompt)
    except APIError as e:
        print(f"API error occurred: {e}. Retrying in 30 seconds...")
        time.sleep(30)
        return get_response(prompt)
    except APITimeoutError as e:
        print(f"Request timed out: {e}. Retrying in 10 seconds...")
        time.sleep(10)
        return get_response(prompt)
    except Exception as e:
        print(f"Unexpected error: {e}. Retrying in 10 seconds...")
        time.sleep(10)
        return get_response(prompt)

# ---- Simulate One Battle ---- #

def simulate_battle(row):
    p1_stats = format_pokemon_stats(row['challenger'], row, '_challenger')
    p2_stats = format_pokemon_stats(row['opponent'], row, '_opponent')

    prompt = (
        "Based on the stats, which Pokémon would win a one-on-one battle?\n\n"
        f"{p1_stats}\nVS\n\n{p2_stats}\n\n"
        "Only respond with the name of the winning Pokémon."
    )

    response = get_response(prompt)
    return response.lower()

# ---- Run All Simulations ---- #

# This should be your DataFrame containing all matchups
# matchups_with_meta = pd.read_csv(...)  # Load your data here

results = []

for idx, row in tqdm(matchups_with_meta.iterrows(), total=len(matchups_with_meta), desc="Simulating battles"):
    print(f"Simulating battle {idx + 1} of {len(matchups_with_meta)}: {row['challenger']} vs {row['opponent']}")
    winner = simulate_battle(row)
    results.append({
        "challenger": row['challenger'],
        "opponent": row['opponent'],
        "winner": winner
    })
    time.sleep(1.5)  # Respect rate limits

# ---- Save Results ---- #

results_df = pd.DataFrame(results)
matchups_with_results = matchups_with_meta.merge(
    results_df,
    on=["challenger", "opponent"],
    how="left"
)
matchups_with_results.head()
matchups_with_results.to_csv(f"/Users/sm9518/Library/CloudStorage/GoogleDrive-sm9518@princeton.edu/My Drive/Classes/Stats-blog/posts/final-project/data/pokemon_battle_results_{model}_{SEED}_{temperature}.csv", index=False)
print(f"\nDone! Winners saved to pokemon_battle_results_{model}_{SEED}_{temperature}.csv.")

Simulating battles:   0%|          | 0/3020 [00:00<?, ?it/s]

Simulating battle 1 of 3020: bulbasaur vs nidorina
Simulating battle 2 of 3020: bulbasaur vs wartortle
Simulating battle 3 of 3020: bulbasaur vs tentacool
Simulating battle 4 of 3020: bulbasaur vs kadabra
Simulating battle 5 of 3020: bulbasaur vs arcanine
Simulating battle 6 of 3020: bulbasaur vs vulpix
Simulating battle 7 of 3020: bulbasaur vs sandslash
Simulating battle 8 of 3020: bulbasaur vs kabutops
Simulating battle 9 of 3020: bulbasaur vs arbok
Simulating battle 10 of 3020: bulbasaur vs weezing
Simulating battle 11 of 3020: bulbasaur vs caterpie
Simulating battle 12 of 3020: bulbasaur vs blastoise
Simulating battle 13 of 3020: bulbasaur vs pikachu
Simulating battle 14 of 3020: bulbasaur vs primeape
Simulating battle 15 of 3020: bulbasaur vs poliwhirl
Simulating battle 16 of 3020: bulbasaur vs lapras
Simulating battle 17 of 3020: bulbasaur vs zapdos
Simulating battle 18 of 3020: bulbasaur vs meowth
Simulating battle 19 of 3020: bulbasaur vs koffing
Simulating battle 20 of 3020: b

KeyboardInterrupt: 